In [45]:

!pip install openai
import openai

import os
import requests

# load_dotenv()

openai.api_key = 'Enter Key Here'

def get_completion(messages, model="gpt-4", temperature=0, max_tokens=300, tools=None):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        tools=tools
    )
    return response.choices[0].message

if __name__ == "__main__":
    # Define the tools that we want to use
    # This is defined in JSON format for the OpenAI API
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_recipes",
                "description": "Get a list of recommended ingredients to make dishes out of, and give examples. The diet should be one of [vegetarian, lacto-vegetarian, ovo-vegetarian, vegan, pescetarian, paleo, primal]. The intolerances should be one of ["", dairy, egg, gluten, grain, peanut, seafood, sesame, shellfish, soy, sulfite, wheat]. If intolerances are not specified in the prompt, just use "". The minCalories shoulbe be >= 50, maxCalories should be <= 800, maxProtein should be <= 100, minFat >= 1, maxFat <= 100. Make sure you specify values for protein, calories, and fat that you think is best suited for them to achieve their goals.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "diet": {
                            "type": "string",
                            "description": "The diet they follow",
                        },
                        "intolerances": {
                            "type": "string",
                            "description": "What they avoid or what they're allergic to",
                        },
                        "minProtein": {
                            "type": "string",
                            "description": "The minimum protein amount in a meal they should consume to achiece their goals",
                        },
                        "minCalories": {
                            "type": "string",
                            "description": "The minimum amount of calories in a meal they should consume to achiece their goals",
                        },
                        "maxCalories": {
                            "type": "string",
                            "description": "The maximum amount of calories in a meal they should consume to achiece their goals",
                        },
                        "minFat": {
                            "type": "string",
                            "description": "The minimum amount of fat in a meal they should consume to achiece their goals",
                        },
                        "maxFat": {
                            "type": "string",
                            "description": "The maximum amount of fat in a meal they should consume to achiece their goals",
                        },
                    },
                    "required": ["diet, intolerances, minProtein, minCalories, maxCalories, minFat, maxFat"],
                },
            },
        }
    ]

    # Pre-load a message to begin the conversation
    msg = "I am skinny and vegeterain, I want to put on weight and build muscle. What should I eat? Also specify the ingredients required."
    messages = [
        {
            "role": "user",
            "content": msg
        }
    ]
    # print(f"User: {msg}\n---")


    response = get_completion(messages, tools=tools)

    # Now, we need to parse the response - the response will contain a TOOL CALL, rather than a completion.
    # The TOOL CALL tells us the function (and appropriate arguments) that the LLM wants to call.
    # This works because OpenAI's API LLMs have been fine-tuned to understand and call functions - other LLMs, such as Llama, do not have this capability.

    # Uncomment the following line to see the response object
    print(response)

    tool_responses = []
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args = eval(tool_call.function.arguments)

        function_call = function_name + "(\'" + (function_args)["diet"] + "\',"

        if "intolerances" in function_args:
            function_call += "\'" + (function_args)["intolerances"] + "\',"
        else:
            function_call += "\'\',"

        function_call += "\'" + (function_args)["minProtein"] + "\',"
        function_call += "\'" + (function_args)["minCalories"] + "\',"
        function_call += "\'" + (function_args)["maxCalories"] + "\',"
        function_call += "\'" + (function_args)["minFat"] + "\',"
        function_call += "\'" + (function_args)["maxFat"] + "\')"

        tool_response = eval(function_call)
        # print(f"Function returns: {tool_response}\n---")
        tool_responses.append({"function_name": function_name, "tool_response": tool_response})

    # Adjust the messages array for next API call
    messages = [
        {
            "role": "user",
            "content": msg
        },
        response,
    ]

    for idx, tool_call in enumerate(response.tool_calls):
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": tool_responses[idx]['function_name'],
                "content": tool_responses[idx]['tool_response'],
            }
        )


    response = get_completion(messages, tools=tools)

    print(f"AI: {response.content}\n---")

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Bsl85HiITBAwVnvOmQnqAPKv', function=Function(arguments='{\n"diet": "vegetarian",\n"minProtein": "20",\n"minCalories": "500",\n"maxCalories": "800",\n"minFat": "10",\n"maxFat": "30"\n}', name='get_recipes'), type='function')])
Dish 1: Mushroom Goat Cheese Baked Eggs
Image: https://spoonacular.com/recipeImages/782619-312x231.png
Calories: 705.703kcal
Protein: 40.1328g
Fat: 25.1264g
Dish 2: Red Lentil Salad
Image: https://spoonacular.com/recipeImages/658070-312x231.jpg
Calories: 547.9kcal
Protein: 25.3012g
Fat: 22.4418g
Dish 3: Zucchini Salad Pita Pockets
Image: https://spoonacular.com/recipeImages/1095849-312x231.jpg
Calories: 601.964kcal
Protein: 20.3393g
Fat: 22.45g
Dish 4: fennel, Peppers, Lettuce Salad
Image: https://spoonacular.com/recipeImages/665831-312x231.jpg
Calories: 565.698kcal
Protein: 23.5647g
Fat: 24.4875g
Dish 5: Herbed Goat Cheese Yogurt Dip w. Car

In [43]:
import requests
def get_recipes(diet, intolerances, minProtein, minCalories, maxCalories, minFat, maxFat) -> str:
    url = "https://api.spoonacular.com/recipes/complexSearch"
    params = {
        "diet": diet,
        "intolerances": intolerances,
        "minProtein": minProtein,
        "minCalories": minCalories,
        "maxCalories": maxCalories,
        "minFat": minFat,
        "maxFat": maxFat,
        "apiKey": "Enter Key Here"
    }

    response = requests.get(url, params=params)
    recipe_info=""
    if response.status_code == 200:
        response_data = response.json()
        for i, recipe in enumerate(response_data["results"], 1):
          dish_title = recipe["title"]
          image_url = recipe["image"]
          calories = recipe["nutrition"]["nutrients"][0]["amount"]
          calories_unit = recipe["nutrition"]["nutrients"][0]["unit"]
          protein = recipe["nutrition"]["nutrients"][1]["amount"]
          protein_unit = recipe["nutrition"]["nutrients"][1]["unit"]
          fat = recipe["nutrition"]["nutrients"][2]["amount"]
          fat_unit = recipe["nutrition"]["nutrients"][2]["unit"]

          # Construct the string format
          recipe_info += f"Dish {i}: {dish_title}\nImage: {image_url}\nCalories: {calories}{calories_unit}\nProtein: {protein}{protein_unit}\nFat: {fat}{fat_unit}\n"
        print(recipe_info)
    else:
        print("Error:", response.status_code)
    return recipe_info


In [ ]:
get_recipes("vegetarian", "gluten", 15, 200, 800, 5, 20)

Dish 1: Cannellini Bean and Asparagus Salad with Mushrooms
Image: https://spoonacular.com/recipeImages/782585-312x231.jpg
Calories: 482.18kcal
Protein: 31.0687g
Fat: 6.33717g
Dish 2: Red Kidney Bean Jambalaya
Image: https://spoonacular.com/recipeImages/782601-312x231.jpg
Calories: 392.803kcal
Protein: 18.1297g
Fat: 6.45348g
Dish 3: Broccoli and Chickpea Rice Salad
Image: https://spoonacular.com/recipeImages/794349-312x231.jpg
Calories: 354.778kcal
Protein: 15.1453g
Fat: 9.77499g
Dish 4: Herbivoracious' White Bean and Kale Soup
Image: https://spoonacular.com/recipeImages/646738-312x231.jpg
Calories: 331.538kcal
Protein: 16.9913g
Fat: 9.97865g
Dish 5: Tomato and lentil soup
Image: https://spoonacular.com/recipeImages/663559-312x231.jpg
Calories: 340.106kcal
Protein: 18.3095g
Fat: 8.07796g
Dish 6: Instant Pot Quinoa Grain Bowl
Image: https://spoonacular.com/recipeImages/982371-312x231.jpg
Calories: 415.567kcal
Protein: 16.0167g
Fat: 9.68368g
Dish 7: Smoky Black Bean Soup With Sweet Potato

"Dish 1: Cannellini Bean and Asparagus Salad with Mushrooms\nImage: https://spoonacular.com/recipeImages/782585-312x231.jpg\nCalories: 482.18kcal\nProtein: 31.0687g\nFat: 6.33717g\nDish 2: Red Kidney Bean Jambalaya\nImage: https://spoonacular.com/recipeImages/782601-312x231.jpg\nCalories: 392.803kcal\nProtein: 18.1297g\nFat: 6.45348g\nDish 3: Broccoli and Chickpea Rice Salad\nImage: https://spoonacular.com/recipeImages/794349-312x231.jpg\nCalories: 354.778kcal\nProtein: 15.1453g\nFat: 9.77499g\nDish 4: Herbivoracious' White Bean and Kale Soup\nImage: https://spoonacular.com/recipeImages/646738-312x231.jpg\nCalories: 331.538kcal\nProtein: 16.9913g\nFat: 9.97865g\nDish 5: Tomato and lentil soup\nImage: https://spoonacular.com/recipeImages/663559-312x231.jpg\nCalories: 340.106kcal\nProtein: 18.3095g\nFat: 8.07796g\nDish 6: Instant Pot Quinoa Grain Bowl\nImage: https://spoonacular.com/recipeImages/982371-312x231.jpg\nCalories: 415.567kcal\nProtein: 16.0167g\nFat: 9.68368g\nDish 7: Smoky Bl